# Creating an Autogewn Agent that talk to Notion MCP



In [7]:
from autogen_ext.tools.mcp import mcp_server_tools, SseServerParams
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent

params = SseServerParams(
    url="https://mcp.notion.com/sse",        # SSE endpoint
    timeout=10.0,                            # HTTP connect timeout
    sse_read_timeout=300.0                   # SSE read timeout
)
tools = await mcp_server_tools(params)  

  + Exception Group Traceback (most recent call last):
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_52955/1210251342.py", line 10, in <module>
  |     tools = await mcp_server_tools(params)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_factory.py", line 193, in mcp_server_tools
  |     async with create_mcp_server_session(server_params) as temp_session:
  |   File "/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/contextlib.py", line 199, in __aenter__
  |     return await anext(self.gen)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_session.py", line 27, in create_mcp_server_session
  |     async with sse_clien

In [6]:
import asyncio
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

async def main() -> None:
    server_params = StdioServerParams(
        command="npx", args=["-y", "@notionhq/notion-mcp-server"]
    )

    # Get all available tools from the server
    tools = await mcp_server_tools(server_params)

    for tool in tools:
        print(f"{tool.name} - {tool.description}")


await main()

/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


FormatNotSupportedError: Unknown format `json` for `icon` in `GeneratedModel`

In [2]:
from mcp.client.auth import OAuthClientProvider, TokenStorage


In [1]:
"""
notion_oauth_client.py
Run once → browser pops → grant access → paste callback → tokens cached.
Subsequent runs reuse the refresh token so no more browser pop-ups.
"""

import asyncio, json, os, webbrowser, aiohttp
from urllib.parse import urlparse, parse_qs
from pydantic import AnyUrl

from mcp.client.auth import OAuthClientProvider, TokenStorage
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession
from autogen_ext.tools.mcp import mcp_server_tools, StreamableHttpServerParams
from mcp.client.auth import (
    OAuthClientProvider,
    TokenStorage
)
from mcp.shared.auth import OAuthClientMetadata
from pydantic import AnyUrl


# ----  Tiny file-based token cache  -----------------------------------------
class FileTokenStore(TokenStorage):
    def __init__(self, path=".notion_oauth_tokens.json"):
        self.path = path
    async def get_tokens(self):         # type: ignore
        if os.path.exists(self.path):
            return json.load(open(self.path))
    async def set_tokens(self, tokens): # type: ignore
        json.dump(tokens, open(self.path, "w"))
    async def get_client_info(self):    # type: ignore
        return None                     # dynamic registration each time
    async def set_client_info(self, info): pass

# ----  Handlers -------------------------------------------------------------
async def handle_redirect(auth_url: str) -> None:
    print(f"Opening browser → {auth_url}")
    webbrowser.open(auth_url)

async def handle_callback() -> tuple[str, str | None]:
    print("Paste the full callback URL you were redirected to:")
    callback_url = input().strip()
    qs = parse_qs(urlparse(callback_url).query)
    return qs["code"][0], qs.get("state", [None])[0]

# ----  Build the auth provider  --------------------------------------------
oauth = OAuthClientProvider(
    server_url="https://mcp.notion.com",
    client_metadata=OAuthClientMetadata(
        client_name="AdmonishNotionMCPClient",
        redirect_uris=[AnyUrl("http://localhost:3000/callback")],
        grant_types=["authorization_code", "refresh_token"],
        response_types=["code"],
        scope="read write"
    ),
    storage=FileTokenStore(),
    redirect_handler=handle_redirect,    # your existing function
    callback_handler=handle_callback     # your existing function
)

# ----  Use it with Streamable HTTP  ----------------------------------------
async def main():
    params = StreamableHttpServerParams(
        url="https://mcp.notion.com/mcp",
        auth_provider=oauth,
        timeout=15.0,
    )
    tools = await mcp_server_tools(params)
    print("✅  Connected! first five tools:")
    for t in tools[:5]:
        print("   -", t.name)

await main()


  + Exception Group Traceback (most recent call last):
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_29958/2683858048.py", line 74, in <module>
  |     await main()
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_29958/2683858048.py", line 69, in main
  |     tools = await mcp_server_tools(params)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_factory.py", line 198, in mcp_server_tools
  |     async with create_mcp_server_session(server_params) as temp_session:
  |   File "/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/contextlib.py", line 217, in __aexit__
  |     await self.gen.athrow(typ, value, traceback)
  |   File "/Users/hugoevers/VScode-projects/ad

In [2]:
from autogen_ext.tools.mcp import mcp_server_tools, StreamableHttpServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
from dotenv import load_dotenv, find_dotenv
import os

# Load environment variables from .env file
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")



params = StreamableHttpServerParams(
    url="https://mcp.notion.com/mcp",
    headers={                               # <- KEY PART
    "Authorization": f"Bearer {os.environ['NOTION_TOKEN']}",
    "Notion-Version": "2022-06-28"      # same header Notion’s README shows
},
    timeout=10.0,
    sse_read_timeout=300.0

)

tools = await mcp_server_tools(params)

# for i, tool in enumerate(tools[:10], 1):
#     tool_name = getattr(tool, 'name', str(tool))
#     tool_desc = getattr(tool, 'description', 'No description')
#     print(f"   {i}. {tool_name}: {tool_desc[:50]}...")


# prompt = "You are an agent that can interact with Notion MCP. You can use the tools provided to perform various tasks related to Notion. Your responses should be concise and relevant to the task at hand. If you need to use a tool, make sure to provide a clear explanation of what you are doing and why."

# agent = AssistantAgent(
#     name="notion-agent",
#     model_client=OpenAIChatCompletionClient(
#         model="gpt-4o-mini", 
#         api_key=OPENAI_API_KEY
#     ),
#     system_message=prompt,
#     tools=tools
# )

  + Exception Group Traceback (most recent call last):
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_29958/2107984663.py", line 26, in <module>
  |     tools = await mcp_server_tools(params)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_factory.py", line 198, in mcp_server_tools
  |     async with create_mcp_server_session(server_params) as temp_session:
  |   File "/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/contextlib.py", line 217, in __aexit__
  |     await self.gen.athrow(typ, value, traceback)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_session.py", line 43, in create_mcp_server_session
  |     async

In [3]:
import asyncio
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

async def main() -> None:
    server_params = StdioServerParams(
        command="npx", args=["-y", "@notionhq/notion-mcp-server"]
    )

    # Get all available tools from the server
    tools = await mcp_server_tools(server_params)

    for tool in tools:
        print(f"{tool.name} - {tool.description}")


await main()

API-get-user - Retrieve a user
Error Responses:
400: 400
API-get-users - List all users
Error Responses:
400: 400
API-get-self - Retrieve your token's bot user
API-post-database-query - Query a database
API-post-search - Search by title
API-get-block-children - Retrieve block children
API-patch-block-children - Append block children
API-retrieve-a-block - Retrieve a block
API-update-a-block - Update a block
API-delete-a-block - Delete a block
API-retrieve-a-page - Retrieve a page
API-patch-page - Update page properties
API-post-page - Create a page
API-create-a-database - Create a database
API-update-a-database - Update a database
API-retrieve-a-database - Retrieve a database
API-retrieve-a-page-property - Retrieve a page property item
API-retrieve-a-comment - Retrieve comments
API-create-a-comment - Create comment


In [2]:
import asyncio, json, os, webbrowser, secrets
from urllib.parse import urlparse, parse_qs
from pydantic import AnyUrl
from mcp.client.auth import OAuthClientProvider, TokenStorage
from mcp.shared.auth import OAuthClientMetadata            # <-- right import
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession
from autogen_ext.tools.mcp import (
    StreamableHttpServerParams,              # only for metadata later
    mcp_server_tools, StreamableHttpMcpToolAdapter
)

In [3]:
import asyncio, json, os, webbrowser, secrets
from urllib.parse import urlparse, parse_qs
from pydantic import AnyUrl
from mcp.client.auth import OAuthClientProvider, TokenStorage
from mcp.shared.auth import OAuthClientMetadata            # <-- right import
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession
from autogen_ext.tools.mcp import (
    StreamableHttpServerParams,              # only for metadata later
    mcp_server_tools, StreamableHttpMcpToolAdapter
)

# ---- simple file cache ---------------------------------------
class FileTokenStore(TokenStorage):
    def __init__(self, path=".notion_tokens.json"): self.path = path
    async def get_tokens(self):  return json.load(open(self.path)) if os.path.exists(self.path) else None
    async def set_tokens(self, t): json.dump(t, open(self.path, "w"))
    async def get_client_info(self): return None
    async def set_client_info(self, _): pass

async def redirect(url: str):          # open consent screen
    webbrowser.open(url); print("Opened browser for OAuth…")

async def callback():                  # copy-paste handler
    cb = input("Paste the full redirect URL: ").strip()
    q = parse_qs(urlparse(cb).query)
    return q["code"][0], q.get("state", [None])[0]

oauth = OAuthClientProvider(
    server_url="https://mcp.notion.com",
    client_metadata=OAuthClientMetadata(
        client_name="Hugo-AutoGen",
        redirect_uris=[AnyUrl("urn:ietf:wg:oauth:2.0:oob")],  # copy-paste
        grant_types=["authorization_code", "refresh_token"],
        response_types=["code"],
        scope="read write"
    ),
    storage=FileTokenStore(),
    redirect_handler=redirect,
    callback_handler=callback
)
async def connect():
    # ① open the connection with auth
    async with streamablehttp_client(
        url="https://mcp.notion.com/mcp",
        auth=oauth,                # <-- key line
        timeout=15.0
    ) as (read, write):

        async with ClientSession(read, write) as sess:
            await sess.initialize()

            # ② load tools through AutoGen helper, but **reusing** the session
            server_meta = StreamableHttpServerParams(url="https://mcp.notion.com/mcp")
            tools = await mcp_server_tools(server_meta, session=sess)

            print("✅ First five tools:")
            for t in tools[:5]:
                print("  -", t.name)

            return tools, sess           # hand these to your agent later

# await connect()



In [2]:
# ----  Handlers for OAuth flow  -------------------------------------------
async def handle_redirect(auth_url: str) -> None:
    print(f"Opening browser → {auth_url}")
    webbrowser.open(auth_url)

async def handle_callback() -> tuple[str, str | None]:
    print("Paste the full callback URL you were redirected to:")
    callback_url = input().strip()
    qs = parse_qs(urlparse(callback_url).query)
    return qs["code"][0], qs.get("state", [None])[0]

In [4]:
import asyncio, json, os, webbrowser, secrets
from urllib.parse import urlparse, parse_qs
from pydantic import AnyUrl
from mcp.client.auth import OAuthClientProvider, TokenStorage
from mcp.shared.auth import OAuthClientMetadata            # <-- right import
from mcp.client.streamable_http import streamablehttp_client
from mcp import ClientSession
from autogen_ext.tools.mcp import (
    StreamableHttpServerParams,              # only for metadata later
    mcp_server_tools, StreamableHttpMcpToolAdapter
)
# ----  FIXED: HTTP server for automatic callback handling  ----------------
from http.server import BaseHTTPRequestHandler, HTTPServer
import threading

oauth_result = {"code": None, "state": None}

class CallbackHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        qs = parse_qs(urlparse(self.path).query)
        oauth_result["code"] = qs.get("code", [None])[0]
        oauth_result["state"] = qs.get("state", [None])[0]
        self.send_response(200)
        self.end_headers()
        self.wfile.write(b"Auth complete. You can close this window.")
    def log_message(self, *args): 
        pass  # silence logs

def start_http_server(port=3000):
    server = HTTPServer(("localhost", port), CallbackHandler)
    threading.Thread(target=server.serve_forever, daemon=True).start()
    return server

async def handle_redirect_fixed(auth_url: str):
    start_http_server(3000)
    print(f"Opening browser → {auth_url}")
    webbrowser.open(auth_url)

async def handle_callback_fixed():
    # Poll until the browser redirects back
    while oauth_result["code"] is None:
        await asyncio.sleep(0.1)
    return oauth_result["code"], oauth_result["state"]

# ----  FIXED: Build the auth provider with loopback URI  ------------------
oauth_fixed = OAuthClientProvider(
    server_url="https://mcp.notion.com",
    client_metadata=OAuthClientMetadata(
        client_name="AdmonishNotionMCPClient",
        redirect_uris=[AnyUrl("http://localhost:3000")],  # Use loopback, not OOB
        grant_types=["authorization_code", "refresh_token"],
        response_types=["code"],
        scope="read write"
    ),
    storage=FileTokenStore(),
    redirect_handler=handle_redirect_fixed,
    callback_handler=handle_callback_fixed
)

# ----  FIXED: Connect with correct session passing and Accept header  -----
async def connect_fixed():
    # Clear any previous OAuth result
    oauth_result["code"] = None
    oauth_result["state"] = None
    
    # Use streamablehttp_client as async context manager with proper headers
    async with streamablehttp_client(
        url="https://mcp.notion.com/mcp",
        auth=oauth_fixed,
        headers={
            "Accept": "application/json,text/event-stream",  # CRITICAL: Required for SSE
            "Notion-Version": "2022-06-28"
        },
        timeout=15.0,
        sse_read_timeout=300.0
    ) as (read_stream, write_stream, get_session_id):
        # Create the ClientSession with the streams
        session = ClientSession(read_stream, write_stream)
        
        # Initialize the session
        await session.initialize()
        
        # FIXED: Pass BOTH server_params AND session to mcp_server_tools
        server_meta = StreamableHttpServerParams(
            url="https://mcp.notion.com/mcp"
        )
        tools = await mcp_server_tools(server_meta, session=session)

        print(f"✅ Connected! Got {len(tools)} tools")
        for i, tool in enumerate(tools[:5], 1):
            print(f"   {i}. {tool.name}: {tool.description[:50]}...")
        
        return tools  # Return the tools for use with AutoGen
        
# Test the connection
tools = await connect_fixed()

Opening browser → https://mcp.notion.com/authorize?response_type=code&client_id=TnTnlfBbH5xuBMte&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2F&state=mFCNDQZECwdCaEWqdaQN6_4FoirAk7rq4k3FQ_nQy-4&code_challenge=ig6YoxTD92W6J4f4tHo3Y5xChhVj8vlP49BpPJIShmY&code_challenge_method=S256&resource=https%3A%2F%2Fmcp.notion.com&scope=read+write


CancelledError: 

In [ ]:
import asyncio, json, os, webbrowser, threading
from urllib.parse import urlparse, parse_qs
from pydantic import AnyUrl
from http.server import BaseHTTPRequestHandler, HTTPServer
from functools import partial

import httpx
from mcp.client.auth import OAuthClientProvider, TokenStorage
from mcp.shared.auth import OAuthClientMetadata
from mcp.client.streamable_http import streamablehttp_client
from mcp.shared._httpx_utils import create_mcp_http_client       # factory helper
from mcp import ClientSession
from autogen_ext.tools.mcp import StreamableHttpServerParams, mcp_server_tools
# --------------------------------------------------------------------------
# 1) tiny token-store (unchanged)
class FileTokenStore(TokenStorage):
    def __init__(self, path=".notion_tokens.json"): self.path = path
    async def get_tokens(self):  return json.load(open(self.path)) if os.path.exists(self.path) else None
    async def set_tokens(self, t): json.dump(t, open(self.path, "w"))
    async def get_client_info(self): return None
    async def set_client_info(self, _): pass
# 2) loop-back OAuth handlers (unchanged – trimmed here for brevity)
oauth_result = {"code": None, "state": None}
class _CB(BaseHTTPRequestHandler):
    def do_GET(self):
        qs = parse_qs(urlparse(self.path).query)
        oauth_result["code"]  = qs.get("code", [None])[0]
        oauth_result["state"] = qs.get("state",[None])[0]
        self.send_response(200); self.end_headers()
        self.wfile.write(b"Auth complete  you can close this tab"); 
    def log_message(self,*a): pass
def _start_server(): HTTPServer(("localhost",3000),_CB).serve_forever()
async def _redirect(url:str): threading.Thread(target=_start_server,daemon=True).start(); webbrowser.open(url)
async def _callback(): 
    import asyncio
    while oauth_result["code"] is None: await asyncio.sleep(0.1)
    return oauth_result["code"], oauth_result["state"]
# 3) OAuth provider
oauth = OAuthClientProvider(
    server_url="https://mcp.notion.com",
    client_metadata=OAuthClientMetadata(
        client_name="Hugo-AutoGen",
        redirect_uris=[AnyUrl("http://localhost:3000")],
        grant_types=["authorization_code","refresh_token"],
        response_types=["code"],
        scope="read write"
    ),
    storage=FileTokenStore(),
    redirect_handler=_redirect,
    callback_handler=_callback
)
# 4) **NEW** httpx client factory that adds Accept header to every request
HEADERS = {
    "Accept": "application/json,text/event-stream",
    "Notion-Version": "2022-06-28",
}
client_factory = partial(create_mcp_http_client, headers=HEADERS)
# 5) connect
async def connect():
    async with streamablehttp_client(
        url="https://mcp.notion.com/mcp",
        auth=oauth,
        headers=HEADERS,
        httpx_client_factory=client_factory,       # ← key line
        timeout=15.0,
        sse_read_timeout=300
    ) as (read, write, _):
        async with ClientSession(read, write) as sess:
            await sess.initialize()
            tools = await mcp_server_tools(
                StreamableHttpServerParams(url="https://mcp.notion.com/mcp"),
                session=sess
            )
            print("✅ Got", len(tools), "tools →", [t.name for t in tools[:5]])
            return tools
await connect()


In [1]:
import asyncio
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

from dotenv import load_dotenv, find_dotenv
import os
# Load environment variables from .env file
load_dotenv(find_dotenv())


server_params = StdioServerParams(
    command="npx", args=["-y", "@notionhq/notion-mcp-server"], env={'NOTION_TOKEN': os.getenv('NOTION_TOKEN') or 'your_notion_token_here'}
)

# Get all available tools from the server
tools = await mcp_server_tools(server_params)



In [9]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
import os


agent = AssistantAgent(
    name="notion_agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini",
                                            api_key=os.getenv("OPENAI_API_KEY")),
    tools=tools,
    system_message=(
        "You can access Notion databases Tasks, Resources, Projects. "
        "Always filter by database_id when retrieving context."
    )
)
message = TextMessage(content="Use the notion tool to retrieve all databases", source="user")
response = await agent.on_messages([message], CancellationToken())

In [10]:
response

Response(chat_message=ToolCallSummaryMessage(id='dbcf7338-a738-43fd-86cb-60e8baaec355', source='notion_agent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 13, 17, 14, 647296, tzinfo=datetime.timezone.utc), content='[{"type": "text", "text": "{\\"object\\":\\"list\\",\\"results\\":[{\\"object\\":\\"database\\",\\"id\\":\\"6e7e25b9-1b9c-4b76-8612-a9b6e0551157\\",\\"cover\\":null,\\"icon\\":null,\\"created_time\\":\\"2023-01-15T14:22:00.000Z\\",\\"created_by\\":{\\"object\\":\\"user\\",\\"id\\":\\"d83ae234-69f8-46aa-bce7-d35175e26ed7\\"},\\"last_edited_by\\":{\\"object\\":\\"user\\",\\"id\\":\\"d83ae234-69f8-46aa-bce7-d35175e26ed7\\"},\\"last_edited_time\\":\\"2023-01-15T14:22:00.000Z\\",\\"title\\":[{\\"type\\":\\"text\\",\\"text\\":{\\"content\\":\\"IO\\",\\"link\\":null},\\"annotations\\":{\\"bold\\":false,\\"italic\\":false,\\"strikethrough\\":false,\\"underline\\":false,\\"code\\":false,\\"color\\":\\"default\\"},\\"plain_text\\":\\"IO\\",\\"href\\":null}

In [8]:
import os, json, asyncio
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

# Ensure your shell has NOTION_TOKEN=ntn_xxx
headers = {
    "Authorization": f"Bearer {os.environ['NOTION_TOKEN']}",
    "Notion-Version": "2022-06-28"
}
env_vars = {
    "OPENAPI_MCP_HEADERS": json.dumps(headers)
}

server_params = StdioServerParams(
    command="npx",
    args=["-y", "@notionhq/notion-mcp-server"],
    env=env_vars
)
tools = await mcp_server_tools(server_params)
async def list_tools():
    tools = await mcp_server_tools(server_params)
    print("Tools:", [t.name for t in tools])

await list_tools()


Tools: ['API-get-user', 'API-get-users', 'API-get-self', 'API-post-database-query', 'API-post-search', 'API-get-block-children', 'API-patch-block-children', 'API-retrieve-a-block', 'API-update-a-block', 'API-delete-a-block', 'API-retrieve-a-page', 'API-patch-page', 'API-post-page', 'API-create-a-database', 'API-update-a-database', 'API-retrieve-a-database', 'API-retrieve-a-page-property', 'API-retrieve-a-comment', 'API-create-a-comment']


In [8]:
from autogen_ext.tools.mcp import StdioServerParams, mcp_server_tools

params = StdioServerParams(
    url="https://mcp.notion.com/sse",
    headers={
        "Authorization": f"Bearer {os.environ['NOTION_TOKEN']}",  # Use
        "Accept": "text/event-stream",       # SSE only
        "Notion-Version": "2022-06-28"
    },
    timeout=15.0,
    sse_read_timeout=300.0
)

tools = await mcp_server_tools(params)
print([t.name for t in tools])


  + Exception Group Traceback (most recent call last):
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/56/3w3mrqc10cj7_lf2zs9gsq600000gn/T/ipykernel_49446/483427993.py", line 14, in <module>
  |     tools = await mcp_server_tools(params)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_factory.py", line 198, in mcp_server_tools
  |     async with create_mcp_server_session(server_params) as temp_session:
  |   File "/Users/hugoevers/.pyenv/versions/3.10.3/lib/python3.10/contextlib.py", line 199, in __aenter__
  |     return await anext(self.gen)
  |   File "/Users/hugoevers/VScode-projects/admonish-1/.venv/lib/python3.10/site-packages/autogen_ext/tools/mcp/_session.py", line 29, in create_mcp_server_session
  |     async with sse_client